In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset
from transformers import pipeline, EarlyStoppingCallback

In [2]:
df = pd.read_csv('basic_stence_train.csv')

df = df[df['stance_label'].isin(['prorussian', 'proukrainian', 'neutral'])]
df = df.dropna(subset=['comment', 'stance_label'])
df = df[df['comment'].str.strip().str.len() > 0]

In [3]:
label2id = {'prorussian': 0, 'neutral': 1, 'proukrainian': 2}
id2label = {0: 'prorussian', 1: 'neutral', 2: 'proukrainian'}

df['label'] = df['stance_label'].map(label2id)


In [4]:
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)
print(f"Train: {len(train_df)}, Val: {len(val_df)}\n")

Train: 13107, Val: 3277



In [5]:
train_dataset = Dataset.from_pandas(train_df[['comment', 'label']])
val_dataset = Dataset.from_pandas(val_df[['comment', 'label']])

In [6]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
def tokenize_function(examples):
    return tokenizer(
        examples['comment'],
        padding='max_length',
        truncation=True,
        max_length=384
    )

In [8]:
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['comment'])
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=['comment'])

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/13107 [00:00<?, ? examples/s]

Map:   0%|          | 0/3277 [00:00<?, ? examples/s]

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {'accuracy': accuracy, 'f1': f1}

In [11]:
training_args = TrainingArguments(
    output_dir='./distilbert-stance-results',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=1.5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=8,
    weight_decay=0.01,
    warmup_steps=500,
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    seed=42,
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer),
    callbacks=[EarlyStoppingCallback(
        early_stopping_patience=3,
        early_stopping_threshold=0.01
    )],
)

/tmp/ipykernel_29009/4105176718.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()
eval_results = trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.582800,0.541757,0.778151,0.767547
2,0.418500,0.502938,0.795545,0.790673
3,0.316600,0.602182,0.787305,0.783946
4,0.192400,0.813543,0.774489,0.771599
5,0.144300,0.988713,0.789136,0.788387


In [13]:
model.save_pretrained('./distilbert-stance-model')
tokenizer.save_pretrained('./distilbert-stance-model')

('./distilbert-stance-model/tokenizer_config.json',
 './distilbert-stance-model/special_tokens_map.json',
 './distilbert-stance-model/vocab.txt',
 './distilbert-stance-model/added_tokens.json',
 './distilbert-stance-model/tokenizer.json')

In [14]:
classifier = pipeline(
    'text-classification',
    model='./distilbert-stance-model',
    device=0 if torch.cuda.is_available() else -1
)

test_comments = [
    "Ruzzia is pure evil! Ukraine has to win!!!",
    "Zelensky is a liar",
    "This war is horrible for both sides",
    "Slava Ukraini!",
    "Russia isn't the bad guy"
]

print("\n\nTest predictions:")
for comment in test_comments:
    pred = classifier(comment)[0]
    print(f"{comment}\n→ {pred['label']} (score: {pred['score']:.3f})\n")


Device set to use cuda:0




Test predictions:
Ruzzia is pure evil! Ukraine has to win!!!
→ prorussian (score: 0.659)

Zelensky is a liar
→ prorussian (score: 0.491)

This war is horrible for both sides
→ prorussian (score: 0.511)

Slava Ukraini!
→ proukrainian (score: 0.805)

Russia isn't the bad guy
→ neutral (score: 0.498)

